In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import torch

In [14]:
p_dataset_path = "./power/power-tr-train.tsv"
p_data = pd.read_csv(p_dataset_path, sep="\t")

# Display basic information
print(p_data.head())
print(p_data.info())

        id                           speaker sex  \
0  tr18146  ca2031caa4032c51980160359953d507   M   
1  tr18147  4cee0addb3c69f6866869b180f90d45f   M   
2  tr18148  b3d7f76d74ec268492f8190ca123a6b2   M   
3  tr18149  722efac7138c8197a9d1e97eed3a8b18   M   
4  tr18150  fcc61122f3553c57ae207adeb1a1af84   M   

                                                text  \
0  Yeni yasama döneminin ülkemiz için, milletimiz...   
1  Sayın Başkan, değerli milletvekilleri; bugün, ...   
2  Sayın Başkanım, öncelikle yüce Meclisin Başkan...   
3  24’üncü Dönem Meclis Başkanlığına seçilmenizde...   
4  Usul tartışmasında 2 kişi lehte 2 kişi aleyhte...   

                                             text_en  label  
0  Mr. President, dear lawmakers, I salute you, a...      0  
1  Mr. President, members of lawmakers, as I spea...      0  
2  Mr. President, I'm here to share with you the ...      0  
3  Mr. President, under the principles determined...      0  
4  Two in favour of two in the legal deb

In [15]:
# Drop rows with missing translations or text fields
p_data = p_data.dropna(subset=['text', 'label'])

# Display class distribution
print(p_data['label'].value_counts())

label
1    8932
0    8452
Name: count, dtype: int64


In [16]:
from sklearn.model_selection import train_test_split
p_train_data, p_test_data = train_test_split(
    p_data, test_size=0.1, stratify=p_data['label'], random_state=42
)
print(f"Training size: {len(p_train_data)}, Test size: {len(p_test_data)}")

Training size: 15645, Test size: 1739


In [23]:
model_name = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
# Convert pandas DataFrame to Hugging Face Dataset
p_train_dataset = Dataset.from_pandas(p_train_data)
p_test_dataset = Dataset.from_pandas(p_test_data)

# Apply tokenization
tokenized_train_p = p_train_dataset.map(tokenize_function, batched=True)
tokenized_test_p = p_test_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns
tokenized_train_p = tokenized_train_p.remove_columns(["text", "__index_level_0__"])
tokenized_test_p = tokenized_test_p.remove_columns(["text", "__index_level_0__"])

# Set format for PyTorch
tokenized_train_p.set_format("torch")
tokenized_test_p.set_format("torch")

Map: 100%|██████████| 1739/1739 [00:01<00:00, 1264.91 examples/s]


In [24]:
print(tokenized_train_p)

Dataset({
    features: ['id', 'speaker', 'sex', 'text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 15645
})


In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
output_dir = 'C:\\Users\\dor_b\\Documents\\results'
log = 'C:\\Users\\dor_b\\Documents\\logs'
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir=log,
    logging_steps=50,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_p,
    eval_dataset=tokenized_test_p,
    tokenizer=tokenizer,
)
trainer.train()

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\dor_b\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\dor_b\AppData\Local\Temp\ipykernel_15780\951444828.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  2%|▏         | 50/2934 [05:24<5:34:35,  6.96s/it]

{'loss': 0.6702, 'grad_norm': 5.592130184173584, 'learning_rate': 2.948875255623722e-05, 'epoch': 0.05}


  3%|▎         | 100/2934 [11:12<5:32:40,  7.04s/it]

{'loss': 0.554, 'grad_norm': 10.038786888122559, 'learning_rate': 2.8977505112474437e-05, 'epoch': 0.1}


  5%|▌         | 150/2934 [17:03<5:31:28,  7.14s/it]

...

100%|██████████| 2934/2934 [5:44:19<00:00,  6.02s/it]

{'eval_loss': 0.3779737651348114, 'eval_runtime': 64.5917, 'eval_samples_per_second': 26.923, 'eval_steps_per_second': 1.688, 'epoch': 3.0}


100%|██████████| 2934/2934 [5:44:34<00:00,  7.05s/it]

{'train_runtime': 20674.1867, 'train_samples_per_second': 2.27, 'train_steps_per_second': 0.142, 'train_loss': 0.29087191157870823, 'epoch': 3.0}


TrainOutput(global_step=2934, training_loss=0.29087191157870823, metrics={'train_runtime': 20674.1867, 'train_samples_per_second': 2.27, 'train_steps_per_second': 0.142, 'total_flos': 1.23491173833216e+16, 'train_loss': 0.29087191157870823, 'epoch': 3.0})

In [26]:
model.save_pretrained("./xlm_roberta_model_power_tr")
tokenizer.save_pretrained("./xlm_roberta_model_power_tr")

('./xlm_roberta_model_power_tr\\tokenizer_config.json',
 './xlm_roberta_model_power_tr\\special_tokens_map.json',
 './xlm_roberta_model_power_tr\\tokenizer.json')

In [ ]:
import transformers
import torch
import huggingface_hub



model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-classification",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
    use_auth_token="your token here",
    trust_remote_code=True
)

def classification_pipeline(speech):

    messages = [
        {"role": "instruction", "content": """Assume you are politician, which will be asked to classify the ideology of the
                                                speakers' party from turkish parliament. Speech will be given as a text. 
                                                In other words, this involves performing binary classification to determine whether
                                                the speaker’s party is governing (0) or oppositon (1). You are not allowed to answer 
                                                anything else besides the number 1 or 0."""},
        {"role": "input", "content": speech},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=100,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.01,
        top_p=0.9,
    )

    return outputs[0]["generated_text"][len(prompt):]



In [ ]:
for index, row in tokenized_test_p.iterrows():
    print(classification_pipeline(row['text']))

In [ ]:
for index, row in tokenized_test_p.iterrows():
    print(classification_pipeline(row['text_en']))